## 子集

[力扣 72](https://leetcode.cn/problems/subsets/)


### 方法一: DFS

In [1]:
class Solution(object):
    ans = []
    def subsets(self, nums):
        """
        :type nums: List[int]
        :rtype: List[List[int]]
        """
        self.ans = [[]]
        self.backtrace(nums, [])
        return self.ans

    def backtrace(self, nums, sub):
        if len(nums) == 0:
            return
        # 有当前元素
        self.ans.append(sub + [nums[0]])
        self.backtrace(nums[1:], sub + [nums[0]])
        # 没有当前元素
        self.backtrace(nums[1:], sub)


### 方法二: BFS

我们留意每次 `nums` 规模扩大时增加的项目。

[] -->  [1]   --> [2], [1, 2]  --> [3], [1,3], [2,3], [1,2,3]

In [2]:
class Solution(object):
    def subsets(self, nums):
        """
        :type nums: List[int]
        :rtype: List[List[int]]
        """
        ans = [[]]
        n = len(nums)
        for i in range(n):
            for j in range(len(ans)):
                ans.append(ans[j] + [nums[i]])
        return ans


## 子集 II

[力扣 78](https://leetcode.cn/problems/subsets-ii/)

### BFS


[] -->(1) [1]  -->(2) [2], [1, 2]  -->(2) [2,2], [1,2,2] -->(2) [2,2,2], [1,2,2,2]

![](../../imgs/78.png)


In [3]:
class Solution(object):
    def subsetsWithDup(self, nums):
        """
        :type nums: List[int]
        :rtype: List[List[int]]
        """
        nums.sort()
        n = len(nums)
        ans = [[], [nums[0]]]
        last_added = [[nums[0]]]
        for i in range(1, n):
            # 如果不同, 则与所有的相组合
            if nums[i] != nums[i-1]:
                last_added = []
                for j in range(len(ans)):
                    last_added.append(ans[j] + [nums[i]])
                ans.extend(last_added)
            # 如果相同了
            else:
                new_last_added = []
                for j in range(len(last_added)):
                    new_last_added.append(last_added[j] + [nums[i]])
                ans.extend(new_last_added)
                last_added = new_last_added
        return ans


## 方法二: 回溯

In [4]:
class Solution(object):
    def subsetsWithDup(self, nums):
        res = []
        path = []
        nums.sort() # 去重需要先对数组进行排序

        def backtracking(nums, startIndex):
            # 终止条件
            res.append(path[:])
            if startIndex == len(nums):
                return

            # for循环
            for i in range(startIndex, len(nums)):
                # 数层去重
                if i > startIndex and nums[i] == nums[i-1]: # 去重
                    continue
                path.append(nums[i])
                backtracking(nums, i+1)
                path.pop()

        backtracking(nums, 0)
        return res

## 组合

[力扣](https://leetcode.cn/problems/combinations/)

![](../../imgs/77.png)

In [5]:
class Solution(object):
    def combine(self, n, k):
        """
        :type n: int
        :type k: int
        :rtype: List[List[int]]
        """
        ans = []
        def backtrace(start, combo, left):
            # 没有足够选择了, 直接返回
            if n+1-start == left:
                ans.append(combo + [i for i in range(start, n+1)])
                return
            # 选择结束, 返回
            if left == 0:
                ans.append(combo)
                return

            backtrace(start+1, combo, left)
            backtrace(start+1, combo + [start], left-1)

        backtrace(1, [], k)
        return ans

## 最大子数组和

[力扣 53](https://leetcode.cn/problems/maximum-subarray/)

### 方法一: 动态规划, 转化为股票买卖问题

转化为股票买卖问题 I

![](../../imgs/53_dp.png)

In [6]:
class Solution(object):
    def maxSubArray(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        n = len(nums)

        for i in range(1, n):
            nums[i] += nums[i-1]
        nums = [0] + nums

        max_price, max_sum = nums[-1], nums[-1]
        i = n-1
        while i >= 0:
            max_sum = max(max_sum, max_price - nums[i])
            max_price = max(max_price, nums[i])
            i -= 1
        return max_sum


### 方法二: 直接动态规划

子问题: dp[i] := 以 `nums[i]` 结尾的最大子序列和为多少？

![](../../imgs/53_dp2.png)

In [7]:
class Solution(object):
    def maxSubArray(self, nums):
        n = len(nums)

        # dp init
        dp = 0
        max_sum = nums[0]

        # dp sweep
        for i in range(1, n):
            dp = max(nums[i], nums[i]+dp)
            max_sum = max(max_sum, dp)

        return max_sum

## 插入区间

[力扣 57.](https://leetcode.cn/problems/insert-interval/)

In [8]:
class Solution(object):
    def findLast(self, intervals, target, start, end):
        l, r = start, end
        while l < r:
            m = (l+r) // 2
            if target < intervals[m][0]:
                r = m - 1
            else:  # 已经满足条件
                if l == m:
                    break
                l = m
        if r < 0:
            return r
        elif target >= intervals[r][0]:
            return r
        else:
            return r-1

    def insert(self, intervals, newInterval):
        """
        :type intervals: List[List[int]]
        :type newInterval: List[int]
        :rtype: List[List[int]]
        """
        n = len(intervals)

        # 1. 在 intervals[0...n-1] 中找到 newInterval[0] >= intervals[i][0] 的最后一个位置 i
        left = self.findLast(intervals, newInterval[0], 0, n-1)

        # 2. 在 intervals[i...n-1] 中找到 newInterval[1] >= intervals[j][0] 的最后一个位置 j
        right = self.findLast(intervals, newInterval[1], max(left, 0), n-1)

        if left == -1:
            # 开头
            left_part = []
            lower = newInterval[0]
        elif left == n-1 and newInterval[0] > intervals[-1][1]:
            left_part = intervals
            lower = newInterval[0]
        else:
            if newInterval[0] <= intervals[left][1]:
                left_part = intervals[:left]
                lower = intervals[left][0]
            else:
                left_part = intervals[:left+1]
                lower = newInterval[0]
        upper = newInterval[1] if right == -1 else max(intervals[right][1], newInterval[1])

        return left_part + [[lower, upper]] + intervals[right+1:]
